In [3]:
from modelClasses import BasicModel, BatchClipEarlyStopD
from analyzerClasses import BasicAnalyzer, CrossoverAnalyzer, CrossoverAnalyzerUpdated
import tableGetter
import ibapi
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
import tensorflow as tf
from time import sleep
from datetime import datetime, date, timedelta, time
from sklearn.preprocessing import MinMaxScaler
import threading
from decimal import *

df = tableGetter.SpyHistory()
currentModel = BasicModel(df)
currentAnalyzer = BasicAnalyzer()

while True:    
    if datetime.today().time() > time(11) and datetime.today().time() < time(14, 30):

        long = False
        short = False
        lastPrice = 0
        totalWin = 0

        class IBapi(EWrapper, EClient):
            def __init__(self):
                EClient.__init__(self, self)
                self.data = [] #Initialize variable to store candle

            def historicalData(self, reqId, bar):
                self.data.append([bar.date, bar.open, bar.close])

            def historicalDataUpdate(self, reqId, bar):
                self.data.append([bar.date, bar.open, bar.close])

            def nextValidId(self, orderId: int):
                super().nextValidId(orderId)
                self.nextorderId = orderId
                print('The next valid order id is: ', self.nextorderId)

            def orderStatus(self, orderId, status, filled, remaining, avgFullPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
                print('orderStatus - orderid:', orderId, 'status:', status, 'filled', filled, 'remaining', remaining, 'lastFillPrice', lastFillPrice)
            
            def openOrder(self, orderId, contract, order, orderState):
                print('openOrder id:', orderId, contract.symbol, contract.secType, '@', contract.exchange, ':', order.action, order.orderType, order.totalQuantity, orderState.status)

            def execDetails(self, reqId, contract, execution):
                print('Order Executed: ', reqId, contract.symbol, contract.secType, contract.currency, execution.execId, execution.orderId, execution.shares, execution.lastLiquidity)
                
            def position(self, account: str, contract: Contract, position: Decimal, avgCost: float):
                super().position(account, contract, position, avgCost)
                print("Position.", "Account:", account, "Symbol:", contract.symbol, "SecType:", contract.secType, "Currency:", contract.currency,"Position:", decimalMaxString(position), "Avg cost:", floatMaxString(avgCost))
                
        def run_loop():
            app.run()

        app = IBapi()
        app.connect('127.0.0.1', 7496, 125)

        #Start the socket in a thread
        api_thread = threading.Thread(target=run_loop, daemon=True)
        api_thread.start()

        sleep(1) #Sleep interval to allow time for connection to server

        #Create contract object
        contract = Contract()
        contract.symbol = 'SPY'
        contract.secType = 'STK'
        contract.exchange = 'SMART'
        contract.currency = 'USD'
                
        #Create order object
        buyOrder = Order()
        buyOrder.action = 'BUY'
        buyOrder.totalQuantity = 100
        buyOrder.orderType = 'MKT'

        sellOrder = Order()
        sellOrder.action = 'SELL'
        sellOrder.totalQuantity = 100
        sellOrder.orderType = 'MKT'

        buyBigOrder = Order()
        buyBigOrder.action = 'BUY'
        buyBigOrder.totalQuantity = 200
        buyBigOrder.orderType = 'MKT'

        sellBigOrder = Order()
        sellBigOrder.action = 'SELL'
        sellBigOrder.totalQuantity = 200
        sellBigOrder.orderType = 'MKT'


        while datetime.today().time() > time(11) and datetime.today().time() < time(14, 30): 
            
            #Request historical candles
            app.data = []
            app.reqHistoricalData(1, contract, '', '1 D', '5 mins', 'TRADES', 1, 2, False, [])
            sleep(5) #sleep to allow enough time for data to be returned
            todaysData = pd.DataFrame(app.data, columns=['DateTime', 'Open', 'Close'])
            try:
                todaysData['DateTime'] = pd.to_datetime(todaysData['DateTime'],unit='s').dt.tz_localize(tz='UTC')
                todaysData['DateTime'] = todaysData['DateTime'].dt.tz_convert(tz='US/Eastern')
                todaysData['Open'] = todaysData['Open'].astype(float)
                todaysData['Close'] = todaysData['Close'].astype(float)
                todaysData['date'] = [d.date() for d in todaysData['DateTime']]
                todaysData['time'] = [d.time() for d in todaysData['DateTime']]
                todaysData['Difference'] = todaysData['Close'] - todaysData['Open']

            except:
                print('error loading data')

            currentTick = todaysData.iloc[-1]
            todaysData = todaysData[:-1]
            todaysData = todaysData.loc[todaysData['time'].isin(df.time.unique())]
            loadedModel = tf.keras.models.load_model('model_'+str(todaysData.iloc[-1]['time']).replace(':','_')+'.keras')
                        
            X = np.array(currentModel.inputScalar.transform(todaysData[currentModel.inputs]))
            input_sequence = X.reshape(1, len(todaysData), currentModel.numInput)
            predicted_sequence = currentModel.outputScalar.inverse_transform(pd.DataFrame(loadedModel.predict(input_sequence)[0], columns=['Difference']))
            
            actionTaken = currentAnalyzer.action(predicted_sequence,todaysData)

            if actionTaken != '':
                if currentAnalyzer.closed:
                    if actionTaken == 'GOLONG':
                        app.placeOrder(app.nextorderId, contract, buyBigOrder)
                        app.nextorderId += 1
                    if actionTaken == 'GOSHORT':
                        app.placeOrder(app.nextorderId, contract, sellBigOrder)
                        app.nextorderId += 1
                    if actionTaken == 'BUY':
                        app.placeOrder(app.nextorderId, contract, buyOrder)
                        app.nextorderId += 1
                    if actionTaken == 'SELL':
                        app.placeOrder(app.nextorderId, contract, sellOrder)
                        app.nextorderId += 1
                else:
                    if actionTaken == 'GOLONG':
                        app.placeOrder(app.nextorderId, contract, buyOrder)
                        app.nextorderId += 1
                    if actionTaken == 'GOSHORT':
                        app.placeOrder(app.nextorderId, contract, sellOrder)
                        app.nextorderId += 1
            

            dfTotal = pd.DataFrame(columns=['Total','Time'])
            Total = 0
            count = 0
            for currentTime in df.time.unique():
                if(todaysData.loc[todaysData['time']==currentTime].empty is False):
                    Total = Total + todaysData.loc[todaysData['time']==currentTime]['Difference'].values[0]
                else:
                    Total = Total + predicted_sequence[count]
                    count = count + 1
                dfTotal.loc[len(dfTotal.index)] = [Total,currentTime] 
            datetimes = [datetime.combine(date.today(), t) for t in dfTotal["Time"]]
            xformatter = mdates.DateFormatter('%H:%M')
            plt.plot(datetimes, dfTotal["Total"], label = "curve 1")
            plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
            plt.show()
            print(currentAnalyzer.totalWin)

            nextUpdate = datetime(datetime.today().year, datetime.today().month, datetime.today().day, todaysData.iloc[-1]['time'].hour, todaysData.iloc[-1]['time'].minute) + timedelta(minutes=10,seconds=5)
            timeToWait = (nextUpdate - datetime.today()).total_seconds()
            print(timeToWait)
            if timeToWait > 0:
                sleep(timeToWait)

        if currentAnalyzer.long:
            app.placeOrder(app.nextorderId, contract, sellOrder)
            app.nextorderId += 1
        if currentAnalyzer.short:
            app.placeOrder(app.nextorderId, contract, buyOrder)
            app.nextorderId += 1

        app.disconnect()

    else:
        # totalWin =  totalWin + abs(lastPrice  - todaysData.iloc[-1]['Close'])
        sleep(5)


KeyboardInterrupt: 